In [ ]:
import time
import json
import numpy as np
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt
import json
import ast
import scipy.stats as stats
import stanza
import math
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
# import SVC classifier
from sklearn.svm import SVC
# import metrics to compute accuracy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [ ]:
final_df = pd.read_csv('./finalwithDate.csv')

In [ ]:
# final_df = final_df.drop(['Unnamed: 0','reason', 'trafficSource.keyword', 'hits'], axis = 1)
final_df

In [ ]:
final_df['norm_country'] = pd.factorize(final_df['geoNetwork.country'])[0]

In [ ]:
final_df['isAnomaly'].fillna(value = 0, inplace = True)
aomalousSessions = final_df[final_df['isAnomaly'] == 1]
len(aomalousSessions)

In [ ]:
final_df['isAnomaly'].fillna(value = 0, inplace = True)
nonAomalousSessions = final_df[final_df['isAnomaly'] == 0]
nonAomalousSessions

In [ ]:
sampled_anomaly_train = aomalousSessions.sample(frac =.50)
sampled_anomaly_test = pd.concat([aomalousSessions, sampled_anomaly_train, sampled_anomaly_train]).drop_duplicates(keep=False)
sampled_anomaly_train

In [ ]:
sampled_nonAnomaly_train = nonAomalousSessions.sample(frac =.10)
sampled_nonAnomaly_test = pd.concat([nonAomalousSessions, sampled_nonAnomaly_train]).drop_duplicates(keep=False)
sampled_nonAnomaly_test

In [ ]:
train_data = pd.concat([sampled_anomaly_train, sampled_nonAnomaly_train])
train_data

In [ ]:
test_data = pd.concat([sampled_nonAnomaly_test, sampled_anomaly_test])

In [ ]:
test_data.to_csv('./testData.csv')

In [ ]:
#SMOTE Application
pd.value_counts(train_data['isAnomaly']).plot.bar()
plt.title('Anomaly class histogram')
plt.xlabel('isAnomaly')
plt.ylabel('Frequency')
train_data['isAnomaly'].value_counts()



In [ ]:
X_train = train_data.drop(['isAnomaly', 'Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
y_train = train_data['isAnomaly']

In [ ]:
X_test = test_data.drop(['isAnomaly', 'Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
y_test = test_data['isAnomaly']
y_test

In [ ]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [ ]:
# instantiate classifier with default hyperparameters
svc=SVC() 


# fit classifier to training set
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
import pickle
from joblib import Parallel, delayed
import joblib
joblib.dump(svc, 'svm.pkl')

In [ ]:
print('Training set score: {:.4f}'.format(svc.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(svc.score(X_test, y_test)))

In [ ]:
# clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(svc, X_train, y_train, cv=5)
# print("%0.2f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))
scores

In [ ]:
#Logistic Regression Starts
logreg = LogisticRegression(solver='liblinear', random_state=0)
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [ ]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

In [ ]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier



# instantiate the classifier 

rfc = RandomForestClassifier(n_estimators=100, random_state=0)



# fit the model

rfc.fit(X_train, y_train)



# Predict the Test set results

y_pred = rfc.predict(X_test)

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))